**Replace with your name and a brief description of the notebook.**

# Homework 15

This homework uses the Green taxi data from Homework 2 and 14.  The 2021 green taxi trip data from NYC Open Data was filtered to only be trips with a pickup date and time on Feb. 3, 2021. 

A congestion surcharge of $2.75 (shown in the `congestion_surcharge` column) is added to all trips that start, end, or pass through Manhattan, south of 96th street. 

Filtered trip data URL: [https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/Feb3_2021_Green_Taxi_Trip_Data.csv](https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/Feb3_2021_Green_Taxi_Trip_Data.csv)

We will try to predict which trips involve tolls using the `congestion_surcharge`, `passenger_count`, `trip_distance`, and `fare_amount` columns.

For questions related to Homework 14, you can use the Homework 14 solutions posted on Blackboard if you did not complete Homework 14.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as smf
from scipy.special import expit
from scipy.stats import logistic

%matplotlib inline

### Question 1: Loading and cleaning the data

a) Load the data into Python.

In [11]:
data = pd.read_csv("https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/Feb3_2021_Green_Taxi_Trip_Data.csv")

1b) Create a new column called `paid_toll` that contains a 1 if the value in the `tolls_amount` column is greater than 0 and a 0 otherwise.  Hint:  See Lab 1, Section 6.

In [12]:
def is_positive(x):
    if (x > 0):
        return 1
    else:
        return 0

In [13]:
data['paid_toll'] = data.tolls_amount.apply(is_positive)

1c) We will only use whether or not a congestion surcharge was added, so create a new column called `manhattan` that contains a 1 if the value in the `congestion_surcharge` column is greater than 0, and 0 otherwise.

In [14]:
data['manhattan'] = data.congestion_surcharge.apply(is_positive)

1d) Create a new DataFrame (or change your current DataFrame) that only contains the columns `manhattan`, `passenger_count`,`trip_distance`, `fare_amount`, and `paid_toll`

In [15]:
data_filtered = data[['manhattan', 'passenger_count', 'trip_distance', 'fare_amount', 'paid_toll']]

### Question 2:  Cleaning data continued

a) Do any of the rows in the DataFrame created in question 1d have missing values?

In [16]:
data_filtered.describe()

,manhattan,passenger_count,trip_distance,fare_amount,paid_toll
count,2281.000000,1297.000000,2281.000000,2281.000000,2281.000000
mean,0.128014,1.121049,49.399842,19.587405,0.076721
std,0.334179,0.559704,2148.602160,14.294597,0.266206
min,0.000000,1.000000,0.000000,-52.000000,0.000000
25%,0.000000,1.000000,1.350000,9.000000,0.000000
50%,0.000000,1.000000,2.530000,16.520000,0.000000
75%,0.000000,1.000000,5.570000,24.810000,0.000000
max,1.000000,6.000000,102620.980000,115.000000,1.000000


**Answer 2a:** Yes. The `passenger_count` column has missing values.

2b) If you found in 2a that some of the rows have missing values, drop all rows with at least one NaN values.

In [17]:
data_filtered_dropna = data_filtered.dropna(how="any")

2c) How many rows did you drop?

In [20]:
data_filtered.shape[0] - data_filtered_dropna.shape[0]

984

In [24]:
data_filtered_dropna

,manhattan,passenger_count,trip_distance,fare_amount,paid_toll
0,1,1.0,2.77,12.5,0
1,0,1.0,1.15,6.5,0
2,0,1.0,1.41,6.0,0
3,0,1.0,6.19,19.5,0
4,0,1.0,2.74,12.0,0
...,...,...,...,...,...
2275,0,2.0,1.32,7.0,0
2276,0,1.0,0.10,12.0,0
2277,0,1.0,2.51,9.5,0
2279,0,1.0,1.53,8.5,0


**Answer 2c:** I dropped 984 rows

### Question 3: Logistic regression model

a) Create a logistic model where `paid_toll` is the dependent variable, and all other columns in your final, cleaned DataFrame from Question 2c are the independent variables.

In [27]:
logit_model = smf.logit('paid_toll ~ manhattan + passenger_count + trip_distance + fare_amount', data = data_filtered_dropna).fit()
logit_model.summary()

Optimization terminated successfully.
         Current function value: 0.089774
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              paid_toll   No. Observations:                 1297
Model:                          Logit   Df Residuals:                     1292
Method:                           MLE   Df Model:                            4
Date:                Thu, 31 Mar 2022   Pseudo R-squ.:                  0.4885
Time:                        23:50:53   Log-Likelihood:                -116.44
converged:                       True   LL-Null:                       -227.64
Covariance Type:            nonrobust   LLR p-value:                 5.674e-47
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -4.1078      0.943     -4.358      0.000      -5.955      -2.260
manhattan          -2.5506      1.000     -2.551      0.011      -4.510      -0.591
passenger_count    -0.8630      0.863     -1.000      0.317      -2.555       0.829
trip_distance       0.2410      0.054      4.450      0.000       0.135       0.347
fare_amount         0.0220      0.017      1.289      0.197      -0.011       0.055
===================================================================================
"""

3b) Recall that the p-value for each variable (in the column P > |z|) is the probability that we would get the coefficient in the `coef` column or one with a greater magnitude, when that coefficient is really 0 (meaning that variable does not affect the dependent variable).  Make a new logistic model with only those variables with a p-value less than 0.05, and compute the summary.

In [28]:
logit_model = smf.logit('paid_toll ~ trip_distance + manhattan', data = data_filtered_dropna).fit()
logit_model.summary()

Optimization terminated successfully.
         Current function value: 0.090955
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              paid_toll   No. Observations:                 1297
Model:                          Logit   Df Residuals:                     1294
Method:                           MLE   Df Model:                            2
Date:                Thu, 31 Mar 2022   Pseudo R-squ.:                  0.4818
Time:                        23:51:17   Log-Likelihood:                -117.97
converged:                       True   LL-Null:                       -227.64
Covariance Type:            nonrobust   LLR p-value:                 2.340e-48
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -4.8808      0.309    -15.775      0.000      -5.487      -4.274
trip_distance     0.3056      0.027     11.315      0.000       0.253       0.358
manhattan        -2.7743      1.070     -2.592      0.010      -4.872      -0.677
=================================================================================
"""

3c)  What is the Pseudo R-squared value for your new logistic model?  What does it suggest about the fit of the model?  How does it compare to the logistic regression model from Homework 14 that only used `trip_distance` as the independent variable?

**Question 3c answer:** The $R^2$ value is $0.4818$. The $R^2$ value from Homework 14 was $0.4466$. This model is slightly more error-prone than the other one.

3d)  What is the equation of your logistic regression model?  The equation below is written in Latex, which is a system for typing math equations.  Double click on this cell, and replace the part of the equation that says "reg" to finish the formula.  "run" this cell (as you would a code cell) when you are finished.

$$y = \frac{1}{1 + e^{-(-4.8808 -0.3056x_1 + 2.7743x_2)}}$$

### Question 4: Confusion matrix

a) Compute the confusion matrix for your second logistic regression model.

In [30]:
confusion_matrix = logit_model.pred_table()
confusion_matrix

array([[1230.,   12.],
       [  29.,   26.]])

4b) What is the accuracy, sensitivity, specificity, and precision of your model?

In [31]:
true_pos = confusion_matrix[0][0]
false_pos = confusion_matrix[1][0]
false_neg = confusion_matrix[0][1]
true_neg = confusion_matrix[1][1]

In [33]:
accuracy = (true_pos + true_neg)/data_filtered_dropna.shape[0]
sensitivity = true_pos/(true_pos + false_neg)
specificity = true_neg/(true_neg + false_pos)
precision = true_pos/(true_pos + false_pos)

In [34]:
print(f"""
Accuracy: {accuracy}
Sensitivity: {sensitivity}
Specificity: {specificity}
Precision: {precision}
""")


Accuracy: 0.9683885890516577
Sensitivity: 0.9903381642512077
Specificity: 0.4727272727272727
Precision: 0.9769658459094519



4c) Based on your answers to 4a and 4b, what are the strengths and weaknesses of this model?

**4c answer:** It is good at classifying positives correctly, but is not so good at classifying true negatives.